# 1.5 Simple Ciphers

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().absolute().parent))

from src.utilities import display_key_inv_table
from src.helpers import CHARACTERS, strip_text, format_ciphertext, format_plaintext, pos, char_at

## Exercises 1.5

### 2. Find the multiplicative inverse (mod 26) of each of the legitimate multiplicative keys: 1, 3, 5, 7, 9, 11, 15, 17, 19, 21, 23, 25.

In [2]:
def inv_26(key: int) -> int:
    return pow(key, -1, 26)

In [3]:
keys = [1, 3, 5, 7, 9, 11, 15, 17, 19, 21, 23, 25]
inv_keys = [inv_26(key) for key in keys]
display_key_inv_table(keys, inv_keys)

<IPython.core.display.Math object>

> **Note:** Questions reordered to complete the enciphering programs first to use for questionss 3. and 4., below.
> Helper source moved to separate Python module, if you're interested the source is [here](https://github.com/dandoug/cryptomath-book/blob/main/src/helpers.py).

### 7. Write a program that enciphers a message using a keyword scheme.

The program should accept as input both a keyword and a message and should output the enciphered message.  Then, enhance the program by adding decipherment capabilities, i.e. given the keyword and an enciphered message, the program should output the deciphered message.

In [4]:
class KeywordCipher:
    """
    Implement the keyword cipher
    """
    def __init__(self, keyword, key_letter):
        self._key_letter = key_letter.lower()
        # prepare the input keyword by lowercasing and removing all non-alphabetic and duplicate characters
        self._keyword = ''.join(dict.fromkeys(strip_text(keyword)))
        # build the encryption table with the keyword and add the rest of the letters that are not in the keyword
        table = self._keyword + ( ''.join(c for c in CHARACTERS if c not in self._keyword))
        # slice the table so that it wraps per the key_letter
        pos = CHARACTERS.index(self._key_letter) # zero-based for slicing
        self._table = table[-pos:] + table[:-pos]

    def encrypt(self, message: str) -> str:
        ciphertext = ''.join(self._table[CHARACTERS.index(c)] for c in strip_text(message))
        return format_ciphertext(ciphertext)

    def decrypt(self, ciphertext: str) -> str:
        plaintext = ''.join(CHARACTERS[self._table.index(c)] for c in strip_text(ciphertext))
        return format_plaintext(plaintext)

### 8. Write a program that enciphers a message using an affine scheme.

The program should accept as input a plaintext message, a multiplicative key, and an additive key and should output the enciphered message.   Then enhance the program by adding decipherment capabilities, i.e., given the additive and multiplicative keys and an enciphered message, the program should output the deciphered message.

In [5]:
class AffineCipher:
    """
    Implement a affine cipher
    """
    def __init__(self, mult_key, add_key):
        self._mult_key = mult_key
        self._add_key = add_key
        self._inv_mult_key = inv_26(mult_key)  # will generate ValueError if key invalid
        self._inv_add_key = (26 - add_key) % 26

    def encrypt(self, message: str) -> str:
        ciphertext = ''.join(char_at((self._mult_key * (pos(c) + self._add_key)) % 26) for c in strip_text(message))
        return format_ciphertext(ciphertext)

    def decrypt(self, ciphertext: str) -> str:
        plaintext = ''.join(char_at((self._inv_mult_key*pos(c) + self._inv_add_key) % 26) for c in strip_text(ciphertext))
        return format_plaintext(plaintext)

class AdditiveCipher(AffineCipher):
    """
    A additive cipher is a subclass of AffineCipher with a multiplicative key = 1.
    """
    def __init__(self, add_key):
        super().__init__(mult_key=1, add_key=add_key)

class MultiplicativeCipher(AffineCipher):
    """
    A multiplicative cipher is a subclass of AffineCipher with an additive key = 0.
    """
    def __init__(self, mult_key):
        super().__init__(mult_key=mult_key, add_key=0)

### 3. Encipher the following message using various methods

> When the government violates the people's rights, insurrection is, for the people and for each portion of the people, the most sacred of the
> rights and most indispensable of duties.

Ignoring puctionation and capitalization, encipher the text using

* *(a)* An additive cipher with key = 16
* *(b)* A multiplicative cipher with key = 17.
* *(c)* An affine cipher with additive key = 24 and multiplicative key = 3.
* *(d)* A keyword cipher with keyword _constitution_ and key letter = $m$.

In [6]:
layfayette_message = (
    "When the government violates the people's rights, insurrection is, for the people and for " +
    "each portion of the people, the most sacred of the rights and most indispensable of duties."
)

#### *(a)* An additive cipher with key = 16

In [7]:
addc = AdditiveCipher(add_key=16)
ctext = addc.encrypt(layfayette_message)
print(ctext)

MXUDJ XUWEL UHDCU DJLYE BQJUI JXUFU EFBUI HYWXJ IYDIK HHUSJ
YEDYI VEHJX UFUEF BUQDT VEHUQ SXFEH JYEDE VJXUF UEFBU JXUCE
IJIQS HUTEV JXUHY WXJIQ DTCEI JYDTY IFUDI QRBUE VTKJY UI


In [8]:
ptext = addc.decrypt(ctext)
print(ptext)

whenthegovernmentviolatesthepeoplesrightsinsurrectionisforth
epeopleandforeachportionofthepeoplethemostsacredoftherightsa
ndmostindispensableofduties


#### *(b)* A multiplicative cipher with key = 17

In [9]:
multc = MultiplicativeCipher(mult_key=17)
ctext = multc.encrypt(layfayette_message)
print(ctext)

AFGDB FGOUJ GTDMG DBJWU VQBGK BFGLG ULVGK TWOFB KWDKS TTGYB
WUDWK XUTBF GLGUL VGQDP XUTGQ YFLUT BWUDU XBFGL GULVG BFGMU
KBKQY TGPUX BFGTW OFBKQ DPMUK BWDPW KLGDK QHVGU XPSBW GK


In [10]:
ptext = multc.decrypt(ctext)
print(ptext)

whenthegovernmentviolatesthepeoplesrightsinsurrectionisforth
epeopleandforeachportionofthepeoplethemostsacredoftherightsa
ndmostindispensableofduties


#### *(c)* An affine cipher with additive key = 24 and multiplicative key = 3

In [11]:
affc = AffineCipher(mult_key=3, add_key=24)
ctext = affc.encrypt(layfayette_message)
print(ctext)

KRIJB RIOMH IVJGI JBHUM DWBIY BRIPI MPDIY VUORB YUJYE VVICB
UMJUY LMVBR IPIMP DIWJF LMVIW CRPMV BUMJM LBRIP IMPDI BRIGM
YBYWC VIFML BRIVU ORBYW JFGMY BUJFU YPIJY WZDIM LFEBU IY


In [12]:
ptext = affc.decrypt(ctext)
print(ptext)

whenthegovernmentviolatesthepeoplesrightsinsurrectionisforth
epeopleandforeachportionofthepeoplethemostsacredoftherightsa
ndmostindispensableofduties


#### *(d)* A keyword cipher with keyword _constitution_ and key letter = $m$

In [13]:
kwc = KeywordCipher(keyword="constitution", key_letter="m")
ctext = kwc.encrypt(layfayette_message)
print(ctext)

EVPOA VPRND PIOCP OADWN ZJAPU AVPSP NSZPU IWRVA UWOUB IIPLA
WNOWU QNIAV PSPNS ZPJOM QNIPJ LVSNI AWNON QAVPS PNSZP AVPCN
UAUJL IPMNQ AVPIW RVAUJ OMCNU AWOMW USPOU JKZPN QMBAW PU


In [14]:
ptext = kwc.decrypt(ctext)
print(ptext)

whenthegovernmentviolatesthepeoplesrightsinsurrectionisforth
epeopleandforeachportionofthepeoplethemostsacredoftherightsa
ndmostindispensableofduties


### 4. Encipher the following message using various methods

> The essence of the free press is the reliable, reasonable, and moral nature of freedom.  The character of the censored press is the nondescript
> confusion of tyranny.

Ignoring puctionation and capitalization, encipher the text using

* *(a)* An additive cipher with key = 6
* *(b)* A multiplicative cipher with key = 11.
* *(c)* An affine cipher with additive key = 2 and multiplicative key = 23.
* *(d)* A keyword cipher with keyword _communist_ and key letter = $z$.

In [15]:
marx_message = (
    "The essence of the free press is the reliable, reasonable, and moral nature of freedom.  " +
    "The character of the censored press is the nondescript confusion of tyranny."
)

#### *(a)* An additive cipher with key = 6

In [16]:
addc = AdditiveCipher(add_key=6)
ctext = addc.encrypt(marx_message)
print(ctext)

ZNKKY YKTIK ULZNK LXKKV XKYYO YZNKX KROGH RKXKG YUTGH RKGTJ
SUXGR TGZAX KULLX KKJUS ZNKIN GXGIZ KXULZ NKIKT YUXKJ VXKYY
OYZNK TUTJK YIXOV ZIUTL AYOUT ULZEX GTTE


In [17]:
ptext = addc.decrypt(ctext)
print(ptext)

theessenceofthefreepressisthereliablereasonableandmoralnatur
eoffreedomthecharacterofthecensoredpressisthenondescriptconf
usionoftyranny


#### *(b)* A multiplicative cipher with key = 11.

In [18]:
multc = MultiplicativeCipher(mult_key=11)
ctext = multc.encrypt(marx_message)
print(ctext)

LJCCA ACXGC INLJC NPCCT PCAAU ALJCP CBUKV BCPCK AIXKV BCKXR
MIPKB XKLWP CINNP CCRIM LJCGJ KPKGL CPINL JCGCX AIPCR TPCAA
UALJC XIXRC AGPUT LGIXN WAUIX INLOP KXXO


In [19]:
ptext = multc.decrypt(ctext)
print(ptext)

theessenceofthefreepressisthereliablereasonableandmoralnatur
eoffreedomthecharacterofthecensoredpressisthenondescriptconf
usionoftyranny


#### *(c)* An affine cipher with additive key = 2 and multiplicative key = 23.

In [20]:
affc = AffineCipher(mult_key=23, add_key=2)
ctext = affc.encrypt(marx_message)
print(ctext)

LVEEO OEDKE ABLVE BREEX REOOS OLVER EJSQN JEREQ OADQN JEQDH
GARQJ DQLIR EABBR EEHAG LVEKV QRQKL ERABL VEKED OAREH XREOO
SOLVE DADHE OKRSX LKADB IOSAD ABLWR QDDW


In [21]:
ptext = affc.decrypt(ctext)
print(ptext)

theessenceofthefreepressisthereliablereasonableandmoralnatur
eoffreedomthecharacterofthecensoredpressisthenondescriptconf
usionoftyranny


#### *(d)* A keyword cipher with keyword _communist_ and key letter = $z$.

In [22]:
kwc = KeywordCipher(keyword="communist", key_letter="z")
ctext = kwc.encrypt(marx_message)
print(ctext)

RAIIQ QIHUI JSRAI SPIIK PIQQB QRAIP IFBOM FIPIO QJHOM FIOHN
GJPOF HORVP IJSSP IINJG RAIUA OPOUR IPJSR AIUIH QJPIN KPIQQ
BQRAI HJHNI QUPBK RUJHS VQBJH JSRZP OHHZ


In [23]:
ptext = kwc.decrypt(ctext)
print(ptext)

theessenceofthefreepressisthereliablereasonableandmoralnatur
eoffreedomthecharacterofthecensoredpressisthenondescriptconf
usionoftyranny
